# Tracking algorithms

Comparing (linear for now) MPC and linear feedback (proportional control).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint
import time

In [ ]:
from dyn.linear_long import LinearLong
from dyn.nonlinear_long import NonlinearLong
from alg.linear_feedback import LinearFeedback
from alg.linear_mpc import LinearMPC

## Tracking: constant reference input, linear feedback, MPC

In [ ]:
# linear dynamics setup
dt = 0.1
x_min = np.array([-np.inf, -np.inf, -np.inf])
x_max = np.array([np.inf, np.inf, np.inf])
u_min = np.array([-5.0])
u_max = np.array([5.0])
tau = 0.5
linear_dyn = LinearLong(dt, x_min, x_max, u_min, u_max, tau)

In [ ]:
# nonlinear dynamics setup
dt = 0.1
x_min = np.array([-np.inf, -np.inf, -np.inf])
x_max = np.array([np.inf, np.inf, np.inf])
u_min = np.array([-6.0])
u_max = np.array([6.0])
m = 1500
c_a = 1.0
g = 9.8
f_r = 0.7
tau = 0.5
r = 0.23
eta_T = 0.7

nonlinear_dyn = NonlinearLong(dt, x_min, x_max, u_min, u_max, 
                              m, c_a, g, f_r, tau, r, eta_T)

In [ ]:
# test the dynamics functions
t_range = np.arange(start=0.0, stop=10.+dt, step=dt)
T = len(t_range)

x_nl_ts = np.zeros((nonlinear_dyn.n, T))
x_nl_ts[2, 0] = nonlinear_dyn.h(x_nl_ts[1, 0])

x_lin_ts = np.zeros((nonlinear_dyn.n, T))
x_lin_ts[:, 0] = np.array([0.0, 0.0, 0.0])
a_des = 1.
u_lin = a_des

# goal: track double integrator trajectory with constant acceleration
def double_int(x, t, u):
    dxdt = np.array(([0, 1], [0, 0])) @ x + np.array([0, 1]) * u
    return dxdt
x0 = np.array([0, 0])
x_dint = odeint(double_int, x0, t_range, args=(u_lin,))
x_dint = x_dint.T

for k in range(1, T):
    x_lin_ts[:, k] = linear_dyn.forward(x_lin_ts[:, k-1], np.array([u_lin]))

    T_des = a_des * m * r / eta_T + nonlinear_dyn.h(x_dint[1, k-1])
    u_nl = T_des
    x_nl_ts[:, k] = nonlinear_dyn.forward(x_nl_ts[:, k-1], np.array([u_nl]))

plt.plot(t_range, x_dint[0, :], label='double int')
plt.plot(t_range, x_lin_ts[0, :], label='linear')
plt.plot(t_range, x_nl_ts[0, :], label='nonlinear')
plt.title('Model comparison: constant input')
plt.xlabel('time [s]')
plt.ylabel('position [m]')
plt.grid()
plt.legend();

Using a constant input like that of the double integrator trajectory will not work to track the double integrator trajectory perfectly. This is due to the time lag. We will try to use linear feedback and MPC for each model to track the double integrator trajectory.


In [ ]:
x_dint = np.r_[x_dint, a_des * np.ones((1, T))]  # reference trajectory

In [ ]:
# linear feedback with the linearized longitudinal model to track the double 
# integrator trajectory

x_ref = x_dint
k_p = 1.
k_v = 2.
k_a = 1.
lin_fbk = LinearFeedback(k_p, k_v, k_a)

# unbounded
x_lfbk_unbounded = np.zeros_like(x_ref)
x_lfbk_unbounded[:, 0] = x_ref[:, 0]
u_lfbk_unbounded = np.zeros(x_ref.shape[1] - 1)

# bounded
x_lfbk_bounded = np.zeros_like(x_ref)
x_lfbk_bounded[:, 0] = x_ref[:, 0]
u_lfbk_bounded = np.zeros(x_ref.shape[1] - 1)
u_min = np.array([-5.0])
u_max = np.array([5.0])

for t in range(T-1):
    unbounded_err = x_lfbk_unbounded[:, t] - x_ref[:, t]
    u = lin_fbk.control(*unbounded_err)
    u_lfbk_unbounded[t] = u
    x_lfbk_unbounded[:, t+1] = linear_dyn.forward(x_lfbk_unbounded[:, t], np.array([u]))

    u = np.clip(u, u_min, u_max)
    u_lfbk_bounded[t] = u
    x_lfbk_bounded[:, t+1] = linear_dyn.forward(x_lfbk_bounded[:, t], u)

In [ ]:
# linear MPC with the linearized longitudinal model to track the double 
# integrator trajectory
z = x_dint
dt = 0.1
x_min = np.array([-np.inf, -np.inf, -np.inf])
x_max = np.array([np.inf, np.inf, np.inf])
u_min = np.array([-5.0])
u_max = np.array([5.0])
tau = 0.5
linear_dyn = LinearLong(dt, x_min, x_max, u_min, u_max, tau)
H = 20  # horizon

Q = np.eye(linear_dyn.n)
Q_f = 10 * np.eye(linear_dyn.n)
R = 0.1 * np.eye(linear_dyn.m)
linear_mpc = LinearMPC(Q, Q_f, R, linear_dyn, x_min, x_max, u_min, u_max, H, 
                       solver='cvx')

start_time = time.time()
x_lin_mpc = np.zeros((linear_dyn.n, T))
u_lin_mpc = np.zeros((linear_dyn.m, T))
for t in range(T-H):
    x_0 = x_lin_mpc[:, t]
    z = x_dint[:, t:t+H+1]
    x_T = z[:, -1]

    u_traj, x_traj, value = linear_mpc.control(x_0, x_T, z)
    u_opt = u_traj[:, 0]
    u_lin_mpc[:, t] = u_opt
    x_lin_mpc[:, t+1] = linear_dyn.forward(x_0, u_opt)

end_time = time.time()
lin_avg_time = (end_time - start_time) / (T - H)
print(f"linear model average time: {lin_avg_time} s")

In [ ]:
fig, ax = plt.subplots(4, 1, sharex=True, figsize=(4, 8))
fig.suptitle('Model tracking comparison')
fig.subplots_adjust(top=0.94)

ax[0].plot(t_range, x_dint[0, :], label='double int ref')
ax[0].plot(t_range[:T-H], x_lin_mpc[0, :T-H], label='linear + mpc')
ax[0].plot(t_range, x_lfbk_unbounded[0, :], label='linear + unbounded fbk')
ax[0].plot(t_range, x_lfbk_bounded[0, :], label='linear + bounded fbk')
ax[0].set_ylabel('position [m]')
ax[0].grid()

ax[1].plot(t_range, x_dint[1, :], label='double int ref')
ax[1].plot(t_range[:T-H], x_lin_mpc[1, :T-H], label='linear + mpc')
ax[1].plot(t_range, x_lfbk_unbounded[1, :], label='linear + unbounded fbk')
ax[1].plot(t_range, x_lfbk_bounded[1, :], label='linear + bounded fbk')
ax[1].set_ylabel('velocity [m/s]')
ax[1].grid()
ax[1].legend(bbox_to_anchor=(1.02, -0.1), loc="center left")

ax[2].plot(t_range, x_dint[2, :], label='double int ref')
ax[2].plot(t_range[:T-H], x_lin_mpc[2, :T-H], label='linear + mpc')
ax[2].plot(t_range, x_lfbk_unbounded[2, :], label='linear + unbounded fbk')
ax[2].plot(t_range, x_lfbk_bounded[2, :], label='linear + bounded fbk')
ax[2].set_ylabel(r'acceleration [m/s$^2$]')
ax[2].grid()

ax[3].plot(t_range, x_dint[2, :], label='double int ref')
ax[3].plot(t_range[:T-H], u_lin_mpc[0, :T-H], label='linear + mpc')
ax[3].plot(t_range[:T-1], u_lfbk_unbounded, label='linear + unbounded fbk')
ax[3].plot(t_range[:T-1], u_lfbk_bounded, label='linear + bounded fbk')
ax[3].set_ylabel(r'input acceleration [m/s$^2$]')
ax[3].set_xlabel('time [s]')
ax[3].grid()

## Tracking: reference step, linear feedback, MPC

In [ ]:
dt = 0.1
t_range = np.arange(start=0.0, stop=20.+dt, step=dt)
T = len(t_range)

# reference trajectory
mask = np.logical_and(t_range > 5., t_range <= 10.)
v_ref = np.zeros(T)
v_ref[t_range <= 5.] = 20.
v_ref[mask] = 20 + 2 * (t_range[mask] - t_range[mask][0])
v_ref[t_range > 10.] = 30.

a_ref = np.zeros(T)
a_ref[mask] = 2.

s_ref = np.zeros(T)
s_ref[t_range <= 5.] = 20 * t_range[t_range <= 5.]
s_ref[mask] = (t_range[mask] - 5)**2 + 20 * (t_range[mask] - 5) + s_ref[t_range <= 5.][-1]
s_ref[t_range > 10.] = 30 * (t_range[t_range > 10.] - 10.) + s_ref[mask][-1]

x_ref = np.r_[s_ref, v_ref, a_ref].reshape((3, -1))

In [ ]:
# linear feedback tracking of reference trajectory
k_p = 1.
k_v = 2.
k_a = 1.
lin_fbk = LinearFeedback(k_p, k_v, k_a)

# unbounded
x_lfbk_unbounded = np.zeros_like(x_ref)
x_lfbk_unbounded[:, 0] = x_ref[:, 0]
u_lfbk_unbounded = np.zeros(x_ref.shape[1] - 1)

# bounded
x_lfbk_bounded = np.zeros_like(x_ref)
x_lfbk_bounded[:, 0] = x_ref[:, 0]
u_lfbk_bounded = np.zeros(x_ref.shape[1] - 1)
u_min = np.array([-5.0])
u_max = np.array([5.0])

for t in range(T-1):
    unbounded_err = x_lfbk_unbounded[:, t] - x_ref[:, t]
    u = lin_fbk.control(*unbounded_err)
    u_lfbk_unbounded[t] = u
    x_lfbk_unbounded[:, t+1] = linear_dyn.forward(x_lfbk_unbounded[:, t], np.array([u]))

    u = np.clip(u, u_min, u_max)
    u_lfbk_bounded[t] = u
    x_lfbk_bounded[:, t+1] = linear_dyn.forward(x_lfbk_bounded[:, t], u)

In [ ]:
# first try linear MPC with the linearized longitudinal model to track the 
# double integrator trajectory
dt = 0.1
x_min = np.array([-np.inf, -np.inf, -np.inf])
x_max = np.array([np.inf, np.inf, np.inf])
u_min = np.array([-5.0])
u_max = np.array([5.0])
tau = 0.5
linear_dyn = LinearLong(dt, x_min, x_max, u_min, u_max, tau)
H = 20  # horizon

Q = np.eye(linear_dyn.n)
Q_f = 10.0 * np.eye(linear_dyn.n)
R = 0.0 * np.eye(linear_dyn.m)
linear_mpc = LinearMPC(Q, Q_f, R, linear_dyn, x_min, x_max, u_min, u_max, H, 
                       solver='cvx')

start_time = time.time()
x_lin_mpc = np.zeros((linear_dyn.n, T))
x_lin_mpc[:, 0] = x_ref[:, 0]
u_lin_mpc = np.zeros((linear_dyn.m, T))
for t in range(T-H):
    x_0 = x_lin_mpc[:, t]
    z = x_ref[:, t:t+H+1]
    x_T = None  # z[:, -1]

    u_traj, x_traj, value = linear_mpc.control(x_0, x_T, z)
    u_opt = u_traj[:, 0]
    u_lin_mpc[:, t] = u_opt
    x_lin_mpc[:, t+1] = linear_dyn.forward(x_0, u_opt)

end_time = time.time()
lin_avg_time = (end_time - start_time) / (T - H)
print(f"linear model average time: {lin_avg_time} s")

In [ ]:
fig, ax = plt.subplots(4, 1, figsize=(4, 8), sharex=True)
fig.suptitle("Platoon leader tracking dynamics: Linear feedback", size=16)
fig.subplots_adjust(top=0.92)

ax[0].plot(t_range, x_ref[0, :], label="reference")
ax[0].plot(t_range, x_lfbk_unbounded[0, :], label="linear feedback - unbounded")
ax[0].plot(t_range, x_lfbk_bounded[0, :], label="linear feedback - bounded")
ax[0].plot(t_range[:T-H], x_lin_mpc[0, :T-H], label="linear mpc")
ax[0].set_ylabel("position [m]")
ax[0].grid()

ax[1].plot(t_range, x_ref[1, :], label="reference")
ax[1].plot(t_range, x_lfbk_unbounded[1, :], label="linear feedback - unbounded")
ax[1].plot(t_range, x_lfbk_bounded[1, :], label="linear feedback - bounded")
ax[1].plot(t_range[:T-H], x_lin_mpc[1, :T-H], label="linear mpc")
ax[1].set_ylabel("velocity [m/s]")
ax[1].grid()
ax[1].legend(bbox_to_anchor=(1.02, -0.1), loc="center left")

ax[2].plot(t_range, x_ref[2, :], label="reference")
ax[2].plot(t_range, x_lfbk_unbounded[2, :], label="linear feedback - unbounded")
ax[2].plot(t_range, x_lfbk_bounded[2, :], label="linear feedback - bounded")
ax[2].plot(t_range[:T-H], x_lin_mpc[2, :T-H], label="linear mpc")
ax[2].set_ylabel(r"acceleration [m/s$^2$]")
ax[2].grid()

ax[3].plot(t_range, x_ref[2, :], label="reference")
ax[3].plot(t_range[:-1], u_lfbk_unbounded[:], label="linear feedback - unbounded")
ax[3].plot(t_range[:-1], u_lfbk_bounded[:], label="linear feedback - bounded")
ax[3].plot(t_range[:T-H], u_lin_mpc[0, :T-H], label="linear mpc")
ax[3].set_xlabel("time [s]")
ax[3].set_ylabel(r"input acceleration [m/s$^2$]")
ax[3].grid()

Note: for each case studied so far, the unbounded and bounded perform the same, 
since there's never a need to clip the input.